In [1]:
from dotenv import load_dotenv
from langchain_mistralai import ChatMistralAI
from langchain_mistralai import MistralAIEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_mistralai import ChatMistralAI
from langchain_chroma import Chroma
load_dotenv()

c:\Users\roopa\miniconda3\envs\agentic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


True

In [2]:
#define vector_store embedding model
embed_model=MistralAIEmbeddings()
chat_model=ChatMistralAI()
vector_store=Chroma(embedding_function=embed_model,persist_directory="rumor_detection",collection_name="facts")

In [3]:
documents=[{
  "id": 11,
  "text": "Lung cancer is the leading cause of cancer-related deaths worldwide.",
  "source": "mock_oncology_journal",
  "publisher": "demo_medical_org",
  "year": 2022,
  "topic": "cancer",
  "credibility": "high",
  "validity": "timeless",
  "language": "en"
},
{
  "id": 12,
  "text": "Breast cancer is the most common cancer diagnosed in women worldwide.",
  "source": "mock_oncology_journal",
  "publisher": "demo_medical_org",
  "year": 2022,
  "topic": "cancer",
  "credibility": "high",
  "validity": "timeless",
  "language": "en"
},
{
  "id": 13,
  "text": "Smoking is a major risk factor for developing several types of cancer, including lung, throat, and mouth cancer.",
  "source": "mock_health_statistics",
  "publisher": "demo_health_org",
  "year": 2021,
  "topic": "cancer",
  "credibility": "high",
  "validity": "timeless",
  "language": "en"
},
{
  "id": 14,
  "text": "Early detection through screening significantly improves the chances of successful treatment for colorectal cancer.",
  "source": "mock_medical_textbook",
  "publisher": "demo_medical_org",
  "year": 2020,
  "topic": "cancer",
  "credibility": "high",
  "validity": "timeless",
  "language": "en"
}

]

In [4]:
langchain_docs = []
for d in documents:
    metadata = d.copy()
    text = metadata.pop("text")     # this becomes embedding text
    doc_id = str(metadata.pop("id"))  # Chroma likes string ids

    langchain_docs.append(
        Document(
            page_content=text,
            metadata=metadata,
            id=doc_id
        )
    )

In [5]:
print(langchain_docs)

[Document(id='11', metadata={'source': 'mock_oncology_journal', 'publisher': 'demo_medical_org', 'year': 2022, 'topic': 'cancer', 'credibility': 'high', 'validity': 'timeless', 'language': 'en'}, page_content='Lung cancer is the leading cause of cancer-related deaths worldwide.'), Document(id='12', metadata={'source': 'mock_oncology_journal', 'publisher': 'demo_medical_org', 'year': 2022, 'topic': 'cancer', 'credibility': 'high', 'validity': 'timeless', 'language': 'en'}, page_content='Breast cancer is the most common cancer diagnosed in women worldwide.'), Document(id='13', metadata={'source': 'mock_health_statistics', 'publisher': 'demo_health_org', 'year': 2021, 'topic': 'cancer', 'credibility': 'high', 'validity': 'timeless', 'language': 'en'}, page_content='Smoking is a major risk factor for developing several types of cancer, including lung, throat, and mouth cancer.'), Document(id='14', metadata={'source': 'mock_medical_textbook', 'publisher': 'demo_medical_org', 'year': 2020, '

In [6]:
vector_store.add_documents(langchain_docs)

['11', '12', '13', '14']

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [14]:
retriever.invoke("whats capital of france")

[Document(id='1', metadata={'credibility': 'high', 'topic': 'geography', 'language': 'en', 'year': 2020, 'source': 'mock_geo_book', 'validity': 'timeless', 'publisher': 'demo_encyclopedia'}, page_content='The capital of France is Paris'),
 Document(id='2', metadata={'credibility': 'high', 'publisher': 'demo_encyclopedia', 'language': 'en', 'topic': 'geography', 'year': 2020, 'validity': 'timeless', 'source': 'mock_geo_book'}, page_content='The capital of Japan is Tokyo'),
 Document(id='5', metadata={'language': 'en', 'validity': 'timeless', 'credibility': 'high', 'year': 1600, 'publisher': 'demo_space_agency', 'source': 'mock_astronomy_archive', 'topic': 'astronomy'}, page_content='Earth revolves around the Sun')]

In [ ]:
def format_docs(docs):
    return "\n\n".join(
        f"FACT: {d.page_content}\n"
        f"credibility: {d.metadata.get('credibility')}\n"
        f"source: {d.metadata.get('source')}"
        for d in docs
    )
format_runnable=RunnableLambda(format_docs)